Imports

In [6]:
import kagglehub
import os
import pandas as pd

Data Downloads

In [3]:
# Craigslist
path_craigslist = kagglehub.dataset_download("michaelbryantds/bay-area-craigslist-rentals")

# Germany
path_germany = kagglehub.dataset_download("corrieaar/apartment-rental-offers-in-germany")

# Poland
path_poland = kagglehub.dataset_download("krzysztofjamroz/apartment-prices-in-poland")

In [5]:
# List CSV files in each dataset folder
craigslist_csvs = [f for f in os.listdir(path_craigslist) if f.endswith('.csv')]
germany_csvs = [f for f in os.listdir(path_germany) if f.endswith('.csv')]
poland_csvs = [f for f in os.listdir(path_poland) if f.endswith('.csv')]

print("Craigslist CSVs:", craigslist_csvs)
print("Germany CSVs:", germany_csvs)
print("Poland CSVs:", poland_csvs)


Craigslist CSVs: ['clean_2000_2018.csv']
Germany CSVs: ['immo_data.csv']
Poland CSVs: ['apartments_pl_2023_08.csv', 'apartments_pl_2023_09.csv', 'apartments_pl_2023_10.csv', 'apartments_pl_2023_11.csv', 'apartments_pl_2023_12.csv', 'apartments_pl_2024_01.csv', 'apartments_pl_2024_02.csv', 'apartments_pl_2024_03.csv', 'apartments_pl_2024_04.csv', 'apartments_pl_2024_05.csv', 'apartments_pl_2024_06.csv', 'apartments_rent_pl_2023_11.csv', 'apartments_rent_pl_2023_12.csv', 'apartments_rent_pl_2024_01.csv', 'apartments_rent_pl_2024_02.csv', 'apartments_rent_pl_2024_03.csv', 'apartments_rent_pl_2024_04.csv', 'apartments_rent_pl_2024_05.csv', 'apartments_rent_pl_2024_06.csv']


In [7]:
# Example: Load first CSV file from each dataset
df_craigslist = pd.read_csv(os.path.join(path_craigslist, craigslist_csvs[0]))
df_germany = pd.read_csv(os.path.join(path_germany, germany_csvs[0]))
df_poland = pd.read_csv(os.path.join(path_poland, poland_csvs[0]))


C:\Users\Mahdi\AppData\Local\Temp\ipykernel_18528\783534574.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_craigslist = pd.read_csv(os.path.join(path_craigslist, craigslist_csvs[0]))


In [ ]:
print(df_craigslist.head())
print(df_germany.head())
print(df_poland.head())